# Analyzing Different Optimization Methods for Solar Raster Co-Alignment
#### Author: Juno Kim

### Table of Contents:
* [Inputs](#inputs)
* [Cleaning Datasets](#clean)
#### Small Dataset
* [Basic Statistics](#basicstats)
    * [Minimize Method](#m1)
    * [Evolve Method](#d1)
    * [Basin Hopping Method](#b1)
* [Comparison](#compare1)
* [Conclusions](#conclude1)
#### Large Dataset
* [Minimize Method](#m2)
* [Evolve Method](#d2)
* [Basin Hopping Method](#b2)
* [Final Conclusions](#conclude2)
    * [Error Analysis](#error)

## Inputs <a class="anchor" id="inputs"></a>

In [1]:
import numpy as np
import os
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import statistics

In [25]:
minimize_filename = "optimized_results.csv"
differential_filename = "evolve_results.csv"
basin_filename = "basin_results.csv"
mass_minimize_filename = "mass_results_minimize.csv"
mass_evolve_filename = "mass_results_evolve.csv"
mass_basin_filename = "mass_results_basin.csv"

In [26]:
minimize_csv = pd.read_csv(minimize_filename)
differential_csv = pd.read_csv(differential_filename)
basin_csv = pd.read_csv(basin_filename)
mass_min_csv = pd.read_csv(mass_minimize_filename)
mass_evolve_csv = pd.read_csv(mass_evolve_filename)
mass_basin_csv = pd.read_csv(mass_basin_filename)

## Cleaning  <a class="anchor" id="clean"></a>

### Removing failed attempts (RESULT = 100000)

In [27]:
test = minimize_csv.copy()

In [28]:
test

,Unnamed: 0,OBSID,AIA_N_GUESS,IRIS_N_GUESS,BLUR_GUESS,OPTIMIZED_AIA,OPTIMIZED_IRIS,OPTIMIZED_BLUR,RESULT
0,0,20221008_030317_3620108077,0.19,0.10,20,0.190000,0.102500,20.000000,1359.916272
1,1,20221008_030317_3620108077,0.19,0.10,25,0.190000,0.104375,25.000000,1359.916272
2,2,20221008_030317_3620108077,0.19,0.15,20,0.192375,0.148125,20.250000,1195.161128
3,3,20221008_030317_3620108077,0.19,0.15,25,0.194137,0.145549,24.862291,1195.161128
4,4,20221008_030317_3620108077,0.19,0.20,20,0.194738,0.200013,19.993490,1210.054771
...,...,...,...,...,...,...,...,...,...
283,283,20220730_125009_3620112077,0.23,0.35,25,0.218500,0.361667,25.833333,68496.925343
284,284,20220730_125009_3620112077,0.23,0.40,20,0.224197,0.420278,20.125000,66851.860670
285,285,20220730_125009_3620112077,0.23,0.40,25,0.230000,0.420000,25.000000,66996.953552
286,286,20220730_125009_3620112077,0.23,0.45,20,0.219352,0.460833,20.185185,66682.547705


In [29]:
test.drop(test[test['RESULT'] == 100000].index, inplace=True)

In [30]:
test

,Unnamed: 0,OBSID,AIA_N_GUESS,IRIS_N_GUESS,BLUR_GUESS,OPTIMIZED_AIA,OPTIMIZED_IRIS,OPTIMIZED_BLUR,RESULT
0,0,20221008_030317_3620108077,0.19,0.10,20,0.190000,0.102500,20.000000,1359.916272
1,1,20221008_030317_3620108077,0.19,0.10,25,0.190000,0.104375,25.000000,1359.916272
2,2,20221008_030317_3620108077,0.19,0.15,20,0.192375,0.148125,20.250000,1195.161128
3,3,20221008_030317_3620108077,0.19,0.15,25,0.194137,0.145549,24.862291,1195.161128
4,4,20221008_030317_3620108077,0.19,0.20,20,0.194738,0.200013,19.993490,1210.054771
...,...,...,...,...,...,...,...,...,...
283,283,20220730_125009_3620112077,0.23,0.35,25,0.218500,0.361667,25.833333,68496.925343
284,284,20220730_125009_3620112077,0.23,0.40,20,0.224197,0.420278,20.125000,66851.860670
285,285,20220730_125009_3620112077,0.23,0.40,25,0.230000,0.420000,25.000000,66996.953552
286,286,20220730_125009_3620112077,0.23,0.45,20,0.219352,0.460833,20.185185,66682.547705


In [32]:
minimize_csv.drop(minimize_csv[minimize_csv['RESULT'] == 100000].index, inplace=True)
differential_csv.drop(differential_csv[differential_csv['RESULT'] == 100000].index, inplace=True)
basin_csv.drop(basin_csv[basin_csv['RESULT'] == 100000].index, inplace=True)
mass_min_csv.drop(mass_min_csv[mass_min_csv['RESULT'] == 100000].index, inplace=True)
mass_evolve_csv.drop(mass_evolve_csv[mass_evolve_csv['RESULT'] == 100000].index, inplace=True)
mass_basin_csv.drop(mass_basin_csv[mass_basin_csv['RESULT'] == 100000].index, inplace=True)

## Basic Stats  <a class="anchor" id="basicstats"></a>

In [74]:
def get_statistics(df, column_name):
    df_copy = df.copy()
    print(f"Mean {column_name}: ", np.round(df_copy[column_name].mean(), 6))
    print(f"Standard Deviation in {column_name}: ", np.round(df_copy[column_name].std(), 6))
    print(f"Median {column_name}: ", np.round(df_copy[column_name].median(), 6))
    print(f"Max {column_name}: ", df_copy[column_name].max())
    print(f"Min {column_name}: ", df_copy[column_name].min())

### Scipy Minimize  <a class="anchor" id="m1"></a>

In [75]:
get_statistics(minimize_csv, 'OPTIMIZED_AIA')

Mean OPTIMIZED_AIA:  0.214907
Standard Deviation in OPTIMIZED_AIA:  0.018199
Median OPTIMIZED_AIA:  0.211495
Max OPTIMIZED_AIA:  0.2567770919067214
Min OPTIMIZED_AIA:  0.1797596450617285


In [76]:
get_statistics(minimize_csv, 'OPTIMIZED_IRIS')

Mean OPTIMIZED_IRIS:  0.281563
Standard Deviation in OPTIMIZED_IRIS:  0.102838
Median OPTIMIZED_IRIS:  0.297338
Max OPTIMIZED_IRIS:  0.4696527777777778
Min OPTIMIZED_IRIS:  0.0887012031321445


In [77]:
get_statistics(minimize_csv, 'OPTIMIZED_BLUR')

Mean OPTIMIZED_BLUR:  22.554918
Standard Deviation in OPTIMIZED_BLUR:  2.777722
Median OPTIMIZED_BLUR:  21.851852
Max OPTIMIZED_BLUR:  32.39605957552203
Min OPTIMIZED_BLUR:  17.49657064471881


In [78]:
get_statistics(minimize_csv, 'RESULT')

Mean RESULT:  19937.174843
Standard Deviation in RESULT:  29735.173754
Median RESULT:  1195.161128
Max RESULT:  71736.27946262652
Min RESULT:  761.3941010239749


### Differential Evolution  <a class="anchor" id="d1"></a>

In [79]:
get_statistics(differential_csv, 'OPTIMIZED_AIA')

Mean OPTIMIZED_AIA:  0.213215
Standard Deviation in OPTIMIZED_AIA:  0.014362
Median OPTIMIZED_AIA:  0.207527
Max OPTIMIZED_AIA:  0.2484994251759107
Min OPTIMIZED_AIA:  0.190424566415833


In [77]:
get_statistics(differential_csv, 'OPTIMIZED_IRIS')

Mean OPTIMIZED_IRIS:  0.285943
Standard Deviation in OPTIMIZED_IRIS:  0.07879
Median OPTIMIZED_IRIS:  0.273237
Max OPTIMIZED_IRIS:  0.4933139405458646
Min OPTIMIZED_IRIS:  0.1558441750128726


In [78]:
get_statistics(differential_csv, 'OPTIMIZED_BLUR')

Mean OPTIMIZED_BLUR:  23.958051
Standard Deviation in OPTIMIZED_BLUR:  2.692976
Median OPTIMIZED_BLUR:  23.480692
Max OPTIMIZED_BLUR:  29.926340017804915
Min OPTIMIZED_BLUR:  20.000243933875165


In [79]:
get_statistics(differential_csv, 'RESULT')

Mean RESULT:  15090.094865
Standard Deviation in RESULT:  25666.14713
Median RESULT:  926.187589
Max RESULT:  66920.1981872239
Min RESULT:  759.1132037351919


### Basin Hopping  <a class="anchor" id="b1"></a>

In [80]:
get_statistics(basin_csv, 'OPTIMIZED_AIA')

Mean OPTIMIZED_AIA:  0.255986
Standard Deviation in OPTIMIZED_AIA:  0.057933
Median OPTIMIZED_AIA:  0.293379
Max OPTIMIZED_AIA:  0.3
Min OPTIMIZED_AIA:  0.1


In [81]:
get_statistics(basin_csv, 'OPTIMIZED_IRIS')

Mean OPTIMIZED_IRIS:  0.318812
Standard Deviation in OPTIMIZED_IRIS:  0.103966
Median OPTIMIZED_IRIS:  0.309099
Max OPTIMIZED_IRIS:  0.5
Min OPTIMIZED_IRIS:  0.1


In [82]:
get_statistics(basin_csv, 'OPTIMIZED_BLUR')

Mean OPTIMIZED_BLUR:  22.753841
Standard Deviation in OPTIMIZED_BLUR:  2.465819
Median OPTIMIZED_BLUR:  22.759985
Max OPTIMIZED_BLUR:  27.63719566254376
Min OPTIMIZED_BLUR:  20.0


In [83]:
get_statistics(basin_csv, 'RESULT')

Mean RESULT:  15603.023768
Standard Deviation in RESULT:  25678.724188
Median RESULT:  1010.165112
Max RESULT:  68160.18627559583
Min RESULT:  763.5993606748762


## Initial Method Comparison  <a class="anchor" id="compare1"></a>

In [80]:
print("Minimize Method")
get_statistics(minimize_csv, 'RESULT')
print('\n')
print("Evolve Method")
get_statistics(differential_csv, 'RESULT')
print('\n')
print("Basin Hopping Method")
get_statistics(basin_csv, 'RESULT')

Minimize Method
Mean RESULT:  19937.174843
Standard Deviation in RESULT:  29735.173754
Median RESULT:  1195.161128
Max RESULT:  71736.27946262652
Min RESULT:  761.3941010239749


Evolve Method
Mean RESULT:  15090.094865
Standard Deviation in RESULT:  25666.14713
Median RESULT:  926.187589
Max RESULT:  66920.1981872239
Min RESULT:  759.1132037351919


Basin Hopping Method
Mean RESULT:  15603.023768
Standard Deviation in RESULT:  25678.724188
Median RESULT:  1010.165112
Max RESULT:  68160.18627559583
Min RESULT:  763.5993606748762


## Conclusions  <a class="anchor" id="conclude1"></a>

- differential evolution has lowest average result + standard deviation
- however, also has highest runtime
- basin hopping has slightly lower runtime and similar results
- by increasing data, can observe effects of runtime + more

## Mass Minimize Results  <a class="anchor" id="m2"></a>

In [20]:
get_statistics(mass_min_csv, 'OPTIMIZED_AIA')

Mean OPTIMIZED_AIA:  0.219832
Standard Deviation in OPTIMIZED_AIA:  0.008833
Median OPTIMIZED_AIA:  0.217859
Max OPTIMIZED_AIA:  0.2370029550540124
Min OPTIMIZED_AIA:  0.2053555777777777


In [21]:
get_statistics(mass_min_csv, 'OPTIMIZED_IRIS')

Mean OPTIMIZED_IRIS:  0.281803
Standard Deviation in OPTIMIZED_IRIS:  0.01168
Median OPTIMIZED_IRIS:  0.281563
Max OPTIMIZED_IRIS:  0.2960362832609954
Min OPTIMIZED_IRIS:  0.248045922974537


In [22]:
get_statistics(mass_min_csv, 'OPTIMIZED_BLUR')

Mean OPTIMIZED_BLUR:  22.744145
Standard Deviation in OPTIMIZED_BLUR:  0.777263
Median OPTIMIZED_BLUR:  22.554918
Max OPTIMIZED_BLUR:  24.4344945
Min OPTIMIZED_BLUR:  21.46110889791666


In [23]:
get_statistics(mass_min_csv, 'RESULT')

Mean RESULT:  22954.710522
Standard Deviation in RESULT:  30657.214185
Median RESULT:  1756.583935
Max RESULT:  68768.85318879073
Min RESULT:  820.8427938170672


In [24]:
get_statistics(mass_min_csv, 'RUNTIME')

Mean RUNTIME:  31.97472
Standard Deviation in RUNTIME:  5.019473
Median RUNTIME:  32.550545
Max RUNTIME:  39.56492686271668
Min RUNTIME:  22.213770151138306


## Mass Evolve Results  <a class="anchor" id="d2"></a>

In [33]:
get_statistics(mass_evolve_csv, 'OPTIMIZED_AIA')

Mean OPTIMIZED_AIA:  0.214265
Standard Deviation in OPTIMIZED_AIA:  0.016473
Median OPTIMIZED_AIA:  0.209904
Max OPTIMIZED_AIA:  0.2433022546416886
Min OPTIMIZED_AIA:  0.1906845572655224


In [34]:
get_statistics(mass_evolve_csv, 'OPTIMIZED_IRIS')

Mean OPTIMIZED_IRIS:  0.241511
Standard Deviation in OPTIMIZED_IRIS:  0.080425
Median OPTIMIZED_IRIS:  0.238677
Max OPTIMIZED_IRIS:  0.476436167707791
Min OPTIMIZED_IRIS:  0.1277123583126644


In [35]:
get_statistics(mass_evolve_csv, 'OPTIMIZED_BLUR')

Mean OPTIMIZED_BLUR:  25.420002
Standard Deviation in OPTIMIZED_BLUR:  2.714367
Median OPTIMIZED_BLUR:  25.853635
Max OPTIMIZED_BLUR:  29.628798483049984
Min OPTIMIZED_BLUR:  21.109765014919176


In [36]:
get_statistics(mass_evolve_csv, 'RESULT')

Mean RESULT:  17977.499662
Standard Deviation in RESULT:  28170.388049
Median RESULT:  1316.046339
Max RESULT:  67301.50300840446
Min RESULT:  124.73625194893684


In [37]:
get_statistics(mass_evolve_csv, 'RUNTIME')

Mean RUNTIME:  1105.618416
Standard Deviation in RUNTIME:  782.050396
Median RUNTIME:  1131.397676
Max RUNTIME:  2434.523035764694
Min RUNTIME:  83.15725898742676


## Mass Basin Results  <a class="anchor" id="b2"></a>

In [10]:
get_statistics(mass_basin_csv, 'OPTIMIZED_AIA')

Mean OPTIMIZED_AIA:  0.21276
Standard Deviation in OPTIMIZED_AIA:  0.087828
Median OPTIMIZED_AIA:  0.243861
Max OPTIMIZED_AIA:  0.3
Min OPTIMIZED_AIA:  0.1


In [12]:
get_statistics(mass_basin_csv, 'OPTIMIZED_IRIS')

Mean OPTIMIZED_IRIS:  0.224188
Standard Deviation in OPTIMIZED_IRIS:  0.122337
Median OPTIMIZED_IRIS:  0.195766
Max OPTIMIZED_IRIS:  0.4559548742528978
Min OPTIMIZED_IRIS:  0.1


In [13]:
get_statistics(mass_basin_csv, 'OPTIMIZED_BLUR')

Mean OPTIMIZED_BLUR:  23.01053
Standard Deviation in OPTIMIZED_BLUR:  0.631446
Median OPTIMIZED_BLUR:  22.897053
Max OPTIMIZED_BLUR:  24.51297952331294
Min OPTIMIZED_BLUR:  21.97742526917077


In [14]:
get_statistics(mass_basin_csv, 'RESULT')

Mean RESULT:  18966.795342
Standard Deviation in RESULT:  28703.666486
Median RESULT:  1399.673941
Max RESULT:  67444.57886331464
Min RESULT:  498.31043492787495


In [19]:
get_statistics(mass_basin_csv, 'RUNTIME')

Mean RUNTIME:  441.655149
Standard Deviation in RUNTIME:  56.754873
Median RUNTIME:  449.365949
Max RUNTIME:  502.5054461956024
Min RUNTIME:  274.91920495033264


## Final Conclusions on Large amounts of Data  <a class="anchor" id="conclude2"></a>

In [43]:
print("Minimize Method")
get_statistics(mass_min_csv, 'RESULT')
print('\n')

print("Evolution Method")
get_statistics(mass_evolve_csv, 'RESULT')
print('\n')

print("Basin Hopping Method")
get_statistics(mass_basin_csv, 'RESULT')

Minimize Method
Mean RESULT:  22954.710522
Standard Deviation in RESULT:  30657.214185
Median RESULT:  1756.583935
Max RESULT:  68768.85318879073
Min RESULT:  820.8427938170672


Evolution Method
Mean RESULT:  17977.499662
Standard Deviation in RESULT:  28170.388049
Median RESULT:  1316.046339
Max RESULT:  67301.50300840446
Min RESULT:  124.73625194893684


Basin Hopping Method
Mean RESULT:  18966.795342
Standard Deviation in RESULT:  28703.666486
Median RESULT:  1399.673941
Max RESULT:  67444.57886331464
Min RESULT:  498.31043492787495


In [45]:
print("Minimize Method")
get_statistics(mass_min_csv, 'RUNTIME')
print('\n')
print("Evolution Method")
get_statistics(mass_evolve_csv, 'RUNTIME')
print('\n')
print("Basin Hopping Method")
get_statistics(mass_basin_csv, 'RUNTIME')

Minimize Method
Mean RUNTIME:  31.97472
Standard Deviation in RUNTIME:  5.019473
Median RUNTIME:  32.550545
Max RUNTIME:  39.56492686271668
Min RUNTIME:  22.213770151138306


Evolution Method
Mean RUNTIME:  1105.618416
Standard Deviation in RUNTIME:  782.050396
Median RUNTIME:  1131.397676
Max RUNTIME:  2434.523035764694
Min RUNTIME:  83.15725898742676


Basin Hopping Method
Mean RUNTIME:  441.655149
Standard Deviation in RUNTIME:  56.754873
Median RUNTIME:  449.365949
Max RUNTIME:  502.5054461956024
Min RUNTIME:  274.91920495033264


In [ ]:
print("Conclusion: Evolution method has lowest mean and median result with middle consistency and lowest minimum result")
print("Additionally, Evolution Method has highest mean and median runtime but low consistency")

Conclusion: Evolution method has lowest mean and median result with middle consistency and lowest minimum result
Additionally, Evolution Method has highest mean and median runtime but low consistency


### Error Analysis - Problematic Data  <a class="anchor" id="error"></a>

In [48]:
error_min_csv = pd.read_csv(mass_minimize_filename)
error_evolve_csv = pd.read_csv(mass_evolve_filename)
error_basin_csv = pd.read_csv(mass_basin_filename)

In [67]:
minl = error_min_csv[error_min_csv['RESULT'] == 100000].OBSID

In [68]:
evol = error_evolve_csv[error_evolve_csv['RESULT'] == 100000].OBSID

In [69]:
basinl = error_basin_csv[error_basin_csv['RESULT'] == 100000].OBSID

In [70]:
obl = set().union(*[minl, evol, basinl])

In [73]:
obl

{'20220904_031019_3620258076',
 '20220904_042426_3620258076',
 '20220904_060426_3620258076',
 '20220909_235026_3620108076',
 '20220910_004926_3620108076',
 '20220910_014826_3620108076',
 '20220916_224528_3620108076',
 '20220917_001728_3620108076',
 '20220917_011728_3620108076',
 '20220923_232326_3620108076',
 '20220924_002201_3620108076',
 '20220924_012036_3620108076',
 '20220924_120813_3600108078',
 '20220924_131932_3600110078',
 '20220924_151811_3600112078',
 '20221001_003126_3620108076',
 '20221001_013026_3620108076',
 '20221001_022926_3620108076',
 '20221006_200315_3620010078',
 '20221006_220405_3620010078',
 '20221007_000455_3620010078',
 '20221007_020545_3620010078',
 '20221008_000720_3620108076',
 '20221008_010555_3620108076',
 '20221008_020430_3620108076',
 '20221015_001926_3620108076',
 '20221015_011801_3620108076',
 '20221015_021636_3620108076',
 '20221016_031633_3620509175',
 '20221016_051533_3600609177',
 '20221018_122703_3600606077',
 '20221018_134113_3600606077',
 '202210

### Preparing Data

In [101]:
t1 = error_min_csv[error_min_csv['RESULT'] <= 50000].OBSID

In [102]:
t2 = error_evolve_csv[error_evolve_csv['RESULT'] == 50000].OBSID

In [103]:
t3 = error_basin_csv[error_basin_csv['RESULT'] == 50000].OBSID

In [104]:
test = set().union(*[t1, t2, t3])

In [105]:
test

{'20220924_021937_3620110077',
 '20221006_135137_3620108077',
 '20221007_050103_3620108077',
 '20221007_135202_3620108077',
 '20221007_211016_3620108077',
 '20221008_030317_3620108077',
 '20221025_195518_3600609177',
 '20221026_071530_3600609177',
 '20221026_170654_3600609177',
 '20221029_065819_3600609177'}